<a href="https://colab.research.google.com/github/IgorGava/Atividade-HTML-JSP/blob/main/AP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**QUIZ AMBIENTAL ECOLÓGICO**

Olá, a API realizada nesta atividade conta com o validador de token que está funcionando, juntamente com endpoint funcionais, a documentação não será realizada.
Os códigos estão totalmente disponíveis no Replit e podem ser executados através do swagger no navegador.

Utilização e métodos de uso:

a utilização será feita pelo swagger através da validação do token,

**primeiro passo:** executar o código main.py pelo replit, e assim irá gerar uma homepage na tela inicial, porém o token inicial não está funcional, clique no link vinculado "API Swagger" e seja direcionado para pagina swagger.

**segundo passo:** após entrar no site, clique no /token e troque as strings pelo login e senha citados abaixo, depois de gerar o token, vá para /perguntas e cole-o na parte de autenticação, o token será validado e após isso será gerado a pergunta.

**terceiro passo:** após descer para a parte das /respostas e clicar em try it out, descreva o tema_questão correto(recomendo copiar e colar), e também a alternativa desejada, após executar verifique o retorno abaixo.

**Login: pepino**
**senha: salada**

Atenciosamente,


In [ ]:
from flask import Flask, request, jsonify, send_from_directory, render_template
from flask_swagger_ui import get_swaggerui_blueprint
import random

app = Flask(__name__)

SWAGGER_URL = '/swagger'
API_URL = '/swag/swagger.json'

swaggerui_blueprint = get_swaggerui_blueprint(
    SWAGGER_URL, API_URL, config={'app_name': "Questionário Ambiental Ecológico"})

app.register_blueprint(swaggerui_blueprint, url_prefix=SWAGGER_URL)

tokens = {}

@app.route('/swag/swagger.json')
def serve_swagger():
    return send_from_directory('swag', 'swagger.json')

def generate_random_hash(length=20):
    random_bytes = bytes(random.getrandbits(8) for _ in range(length // 2))
    return random_bytes.hex()

@app.route('/painel')
def painel():
    return render_template('index.html')

@app.route('/token', methods=['POST'])
def register():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')

    if username == "pepino" and password == "salada":
        token = generate_random_hash(20)
        tokens[username] = token
        return jsonify({"token": token})
    else:
        return jsonify({"erro": "Usuário ou senha incorretos"}), 401

def check_token(token):
    return token in tokens.values()

materias = {
    'Preservação Ambiental': {
        'perguntas': {
            1: {
                'pergunta': 'O que é uma pegada ecológica?',
                'alternativas': {
                    'a': 'Quantidade de água que uma pessoa consome',
                    'b': 'Quantidade de terra necessária para manter o estilo de vida de uma pessoa',
                    'c': 'Tipo de sapato que uma pessoa usa',
                    'd': 'Quantidade de lixo que uma pessoa produz'
                },
                'resposta_correta': 'b'
            },
            2: {
                'pergunta': 'O que é a camada de ozônio?',
                'alternativas': {
                    'a': 'Uma camada de poluição nas cidades',
                    'b': 'Um tipo de nuvem',
                    'c': 'Uma camada de gás que protege a Terra dos raios ultravioleta do sol',
                    'd': 'Uma camada de oxigênio ao redor da Terra'
                },
                'resposta_correta': 'c'
            },
            # Adicione mais perguntas conforme necessário
        }
    },
    'Reciclagem e Gestão de Resíduos': {
        'perguntas': {
            1: {
                'pergunta': 'O que é compostagem?',
                'alternativas': {
                    'a': 'Um tipo de reciclagem de plástico',
                    'b': 'Processo de queimar lixo',
                    'c': 'Transformação de matéria orgânica em adubo',
                    'd': 'Separação do lixo seco do molhado'
                },
                'resposta_correta': 'c'
            },
            2: {
                'pergunta': 'Quais dos seguintes materiais é biodegradável?',
                'alternativas': {
                    'a': 'Garrafa PET',
                    'b': 'Casca de banana',
                    'c': 'Saco de plástico',
                    'd': 'Lata de alumínio'
                },
                'resposta_correta': 'b'
            },
            # Adicione mais perguntas conforme necessário
        }
    },
    # Adicione mais matérias conforme necessário
}

@app.route('/pergunta', methods=['GET'])
def get_pergunta():
    token_received = request.headers.get('Authorization')
    if token_received and check_token(token_received):
        nome_materia = random.choice(list(materias.keys()))
        materia = materias[nome_materia]
        id_questao = random.choice(list(materia['perguntas'].keys()))
        pergunta = materia['perguntas'][id_questao]

        resposta = {
            'nome_materia': nome_materia,
            'id_questao': id_questao,
            'pergunta': pergunta['pergunta'],
            'alternativas': pergunta['alternativas']
        }
        return jsonify(resposta)
    else:
        return jsonify({"erro": "Token inválido ou ausente"}), 403

@app.route('/resposta', methods=['POST'])
def post_resposta():
    token_received = request.headers.get('Authorization')
    if token_received and check_token(token_received):
        dados = request.get_json()
        nome_materia = dados.get('nome_materia')
        id_questao = dados.get('id_questao')
        alternativa_usuario = dados.get('alternativa')

        if nome_materia not in materias:
            return jsonify({'erro': 'Matéria não encontrada'}), 404

        materia = materias[nome_materia]
        if id_questao not in materia['perguntas']:
            return jsonify({'erro': 'Pergunta não encontrada'}), 404

        pergunta = materia['perguntas'][id_questao]
        if alternativa_usuario == pergunta['resposta_correta']:
            return jsonify({'resultado': 'correto'})
        else:
            return jsonify({'resultado': 'errado'})
    else:
        return jsonify({"erro": "Token inválido ou ausente"}), 403

@app.route('/')
def homepage():
    return '''<h1>Quiz Ambiental Ecológico</h1>
              <h2> primeiro copie o link: "https://assuredvirtuousminimalsystem.igorgava1.repl.co/swagger" e abra uma nova guia no google, tendo em vista que não roda na interface replit.</h2>

              <h2> após isso siga para a validação do token e a resposta das perguntas. </h2>
              <h2> mais detalhes no link </h2>:
              <h2> https://colab.research.google.com/drive/1HuRF8BbPwONgRFuWTL9Cvdx_CJO0RcF6?authuser=1#scrollTo=uJcumIgoP7Qf </h2>
          '''


if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0')


ModuleNotFoundError: ignored

In [ ]:
{
  "swagger": "2.0",
  "info": {
    "version": "1.0.0",
    "title": "API de Quiz",
    "description": "Uma API para testar seus conhecimentos em anatomia"
  },
  "host": "assuredvirtuousminimalsystem.igorgava1.repl.co",
  "schemes": ["https"],
  "paths": {
    "/token": {
      "post": {
        "summary": "Obter Token",
        "description": "Autentica o usuário e retorna um token para ser usado nas demais requisições",
        "parameters": [
          {
            "in": "body",
            "name": "body",
            "description": "Dados de login",
            "required": true,
            "schema": {
              "type": "object",
              "properties": {
                "username": {
                  "type": "string"
                },
                "password": {
                  "type": "string"
                }
              }
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Token gerado com sucesso",
            "schema": {
              "type": "object",
              "properties": {
                "token": {
                  "type": "string"
                }
              }
            }
          },
          "401": {
            "description": "Usuário ou senha incorretos"
          }
        }
      }
    },
    "/pergunta": {
      "get": {
        "summary": "Obter uma pergunta",
        "description": "Retorna uma pergunta aleatória de anatomia",
        "parameters": [
          {
            "in": "header",
            "name": "Authorization",
            "type": "string",
            "description": "Token de autenticação",
            "required": true
          }
        ],
        "responses": {
          "200": {
            "description": "Pergunta retornada com sucesso",
            "schema": {
              "type": "object",
              "properties": {
                "tema_questao": {
                  "type": "string"
                },
                "id_questao": {
                  "type": "integer"
                },
                "pergunta": {
                  "type": "string"
                },
                "alternativas": {
                  "type": "object",
                  "properties": {
                    "a": {
                      "type": "string"
                    },
                    "b": {
                      "type": "string"
                    },
                    "c": {
                      "type": "string"
                    },
                    "d": {
                      "type": "string"
                    }
                  }
                }
              }
            }
          },
          "403": {
            "description": "Token inválido ou ausente"
          }
        }
      }
    },
    "/resposta": {
      "post": {
        "summary": "Submeter uma resposta",
        "description": "Recebe a resposta do usuário e informa se está correta ou não",
        "parameters": [
          {
            "in": "header",
            "name": "Authorization",
            "type": "string",
            "description": "Token de autenticação",
            "required": true
          },
          {
            "in": "body",
            "name": "body",
            "description": "Resposta do usuário",
            "required": true,
            "schema": {
              "type": "object",
              "properties": {
                "tema_questao": {
                  "type": "string"
                },
                "id_questao": {
                  "type": "integer"
                },
                "alternativa": {
                  "type": "string"
                }
              },
              "required": ["tema_questao", "id_questao", "alternativa"]
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Resposta recebida com sucesso",
            "schema": {
              "type": "object",
              "properties": {
                "resultado": {
                  "type": "string"
                }
              }
            }
          },
          "403": {
            "description": "Token inválido ou ausente"
          }
        }
      }
    }
  }
}


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Login</title>
  <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
  <style>
    body {
      display: flex;
      justify-content: center;
      align-items: center;
      height: 100vh;
      background-color: #f8f9fa;
    }
    .login-form {
      width: 100%;
      max-width: 330px;
      padding: 15px;
      margin: auto;
    }
  </style>
  <script>
    function getToken() {
      var username = document.getElementById('username').value;
      var password = document.getElementById('password').value;

      fetch('/token', {
        method: 'POST',
        headers: {
          'Content-Type': 'application/json'
        },
        body: JSON.stringify({
          username: username,
          password: password
        })
      })
      .then(response => response.json())
      .then(data => {
        if(data.token) {
          document.getElementById('token').innerText = 'Token: ' + data.token;
          document.getElementById('token').className = 'alert alert-success mt-3';
        } else {
          document.getElementById('token').innerText = 'Falha ao gerar token. Verifique o usuário e senha.';
          document.getElementById('token').className = 'alert alert-danger mt-3';
        }
      })
      .catch(error => console.error('Error:', error));
    }
  </script>
</head>
<body>
  <div class="login-form text-center">
    <h1 class="h3 mb-3 font-weight-normal">Por favor, faça login</h1>
    <div class="form-group">
      <label for="username" class="sr-only">Usuário</label>
      <input type="text" id="username" class="form-control" placeholder="Usuário" required autofocus>
    </div>
    <div class="form-group">
      <label for="password" class="sr-only">Senha</label>
      <input type="password" id="password" class="form-control" placeholder="Senha" required>
    </div>
    <button class="btn btn-lg btn-primary btn-block" type="button" onclick="getToken()">Entrar</button>
    <p id="token" class="mt-3"></p>
  </div>
</body>
</html>
